In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Step 1: Load the Digits dataset
digits = load_digits()
X = digits.data
y = digits.target

# Step 2: Preprocess the data
# Standardize the feature data (important for Logistic Regression)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Step 3: Initialize the model and track batch accuracies over time
model = LogisticRegression(max_iter=200)
batch_accuracies = []

# Define batch size (divide data into 5 parts)
batch_size = len(X_train) // 5

# Step 4: Train the model on progressively larger batches of data
for i in range(1, 6):
    # Training on the first i batches of data
    X_train_batch = X_train[:i * batch_size]
    y_train_batch = y_train[:i * batch_size]
    
    model.fit(X_train_batch, y_train_batch)  # Train the model
    
    # Predict on the test set and calculate accuracy
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    batch_accuracies.append(acc)

# Step 5: Calculate Progressive Learning Impact (PLI)
# PLI is the average rate of improvement in accuracy between batches
improvement = []
for i in range(1, len(batch_accuracies)):
    improvement.append(batch_accuracies[i] - batch_accuracies[i - 1])

# Calculate the average improvement (PLI)
PLI = np.mean(improvement)

# Step 6: Compare PLI with traditional accuracy
final_accuracy = batch_accuracies[-1]

# Show the results in a clean format
results_df = pd.DataFrame({
    'Batch': range(1, 6),
    'Accuracy': batch_accuracies,
    'Improvement': [batch_accuracies[i] - batch_accuracies[i-1] if i > 0 else 'N/A' for i in range(5)]
})

# Final output of the model's performance
print("Progressive Learning Impact (PLI): {:.4f}".format(PLI))
print("Final Test Accuracy (Traditional Metric): {:.4f}".format(final_accuracy))

# Display the result as a table
print("\nAccuracy and Improvement per Batch:")
print(results_df)

# Plot the learning curve for better understanding
plt.plot(range(1, 6), batch_accuracies, marker='o', label="Accuracy")
plt.xlabel('Batch number')
plt.ylabel('Accuracy')
plt.title('Model Performance with Progressive Learning')
plt.xticks(range(1, 6))
plt.grid(True)
plt.legend()
plt.show()

# Interpretation of PLI
if PLI > 0.01:
    print("The model shows strong progressive learning, improving continuously.")
else:
    print("The model shows slow or limited progressive learning.")
